In [98]:
from nltk import FreqDist
from nltk.corpus.reader.api import CorpusReader
from nltk.data import LazyLoader
from nltk.tokenize import TreebankWordTokenizer
from nltk.util import AbstractLazySequence, LazyMap, LazyConcatenation
from pymongo import MongoClient
import bs4

import pandas as pd
import os
import json
from pymystem3 import Mystem
import pymorphy2
from pprint import pprint
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from sklearn.base import BaseEstimator, TransformerMixin
from IPython.core.display import display, HTML
import re
from nltk.stem.snowball import SnowballStemmer

In [131]:
raw_data_path = os.path.join("..", "..", "data", "raw")
processed_data_path = os.path.join("..", "..", "data", "processed")
external_data_path = os.path.join("..", "..", "data", "external")
models_path = os.path.join("..", "..", "models")
experiments_path = os.path.join("..", "..", "experiments")

In [3]:
df_processed_documents =  pd.read_csv(os.path.join(raw_data_path, "publicru-dataset-2020-03-16", "lib_public_document.csv"), index_col="id")

In [4]:
df = df_processed_documents[:100]

In [5]:
df.head()

,content_api_id,title,annotation,body,pages,authors,size,last_modified,issue_id,pages_visual
id,,,,,,,,,,
325,182582609,Эффект слабого рубля,Мы взяли две компании с сопоставимой выручкой ...,<body><p><b>Мы взяли две компании с сопоставим...,"[70, 71]",NaN,40754,2016-11-24 13:15:42.057148+00:00,9,"['70', '71']"
21225,175907949,РОССТАТ,Индексы потребительских цен на товары и услуги...,<body><p><b>Индексы потребительских цен на тов...,[4],NaN,28483,2016-11-24 23:46:59.007647+00:00,599,['4']
21226,175907948,Неотложка с планшетом,ЗДОРОВЬЕ . В системе ЕМИАС зарегистрировались ...,<body><p><b>ЗДОРОВЬЕ</b> </p>\n<p>В системе ЕМ...,[4],Сергей Жуков,30346,2016-11-24 23:46:59.090845+00:00,599,['4']
1638,183704356,ПРОГНОЗЫ НОМЕРА,К 2017-2018 годам ныне растущий агрокомплекс с...,<body><p>К 2017-2018 годам ныне растущий агрок...,[5],NaN,818,2016-11-24 14:16:32.322323+00:00,42,['5']
2,154615279,ТАНЕЦ НЕНАСТОЯЩЕГО ЛЕБЕДЯ,ПЕРЕД НАМИ КАК БУДТО СЦЕНА ИЗ БАЛЕТА: ДВА МИСТ...,<body><p>ПЕРЕД НАМИ КАК БУДТО СЦЕНА ИЗ БАЛЕТА:...,"[60, 61]",NaN,44550,2016-11-24 13:01:38.412930+00:00,1,"['60', '61']"


In [28]:
m = Mystem()

In [32]:
result = m.analyze("Связной")

In [38]:
result

[{'analysis': [{'lex': 'связной', 'wt': 0.5662913401, 'gr': 'S,муж,од=им,ед'}],
  'text': 'Связной'},
 {'text': '\n'}]

In [41]:
if "analysis" in result[0]:
    grammemes = result[0]["analysis"][0]["gr"]
    print(grammemes.split(sep=','))

['S', 'муж', 'од=им', 'ед']


In [29]:
text = "Иван Иванович Иванов приехал в Санкт-Петербург"
for word in text.split(" "):
    with m.analyze(word) as result:
        print(result[0].form, 
             "({0})".format(result[0]),
             result[0].stem_grammemes, 
             result[0].flex_grammemes
        )

AttributeError: __enter__

In [9]:
%%timeit

text = "Красивая мама красиво мыла раму"
lemmas = m.lemmatize(text)

358 µs ± 44.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [12]:
text = "Красивая мама красиво мыла раму"
lemmas = m.lemmatize(text)
print("lemmas:", ''.join(lemmas))
print("full info:", json.dumps(m.analyze(text), ensure_ascii=False))

lemmas: красивый мама красиво мыть рама

full info: [{"analysis": [{"lex": "красивый", "wt": 1, "gr": "A=им,ед,полн,жен"}], "text": "Красивая"}, {"text": " "}, {"analysis": [{"lex": "мама", "wt": 1, "gr": "S,жен,од=им,ед"}], "text": "мама"}, {"text": " "}, {"analysis": [{"lex": "красиво", "wt": 0.8149252476, "gr": "ADV="}], "text": "красиво"}, {"text": " "}, {"analysis": [{"lex": "мыть", "wt": 0.441520999, "gr": "V,несов,пе=прош,ед,изъяв,жен"}], "text": "мыла"}, {"text": " "}, {"analysis": [{"lex": "рама", "wt": 0.9993591156, "gr": "S,жен,неод=вин,ед"}], "text": "раму"}, {"text": "\n"}]


In [13]:
titles = [ row["title"] for _, row in df.iterrows()]
posts = [ row["body"] for _, row in df.iterrows()]

In [14]:
i=2
print(titles[i], "\n\n")
display(HTML(posts[i]))

Неотложка с планшетом 




In [132]:
def paras(post):
    paras_tags = ["h1", "h2", "h3", "h4", "h5", "h6", "p", "li"]
    soup = bs4.BeautifulSoup(post, "lxml")
    for element in soup.find_all(paras_tags):
        yield element.text
    soup.decompose()
    
def sents(post):
    for para in paras(post):
        for sentence in sent_tokenize(para):
            yield sentence
            
def load_stop_words():
    with open(os.path.join(external_data_path, 'russian_stop_words.txt'), 'r') as handle:
        stop_words_ru = [line.rstrip('\n') for line in handle]
    with open(os.path.join(external_data_path, 'english_stop_words.txt'), 'r') as handle:
        stop_words_en = [line.rstrip('\n') for line in handle]
    stop_words = set(stop_words_ru + stop_words_en)
    return stop_words

In [112]:
for para in paras(posts[0]):
    for sent in sents(para):
        pprint(sent)

('Мы взяли две компании с сопоставимой выручкой и оценили влияние курса рубля '
 'на их финансы.')
('Первый участник - это производитель удобрений - «ФосАгро», который большую '
 'часть выручки получает от экспорта.')
('Второй участник ориентирован на внутренний рынок - это компания «М.видео», '
 'продающая электронику и технику.')
('В качестве начала отчета возьмем 2013 год, когда курс рубля к доллару '
 'оставался стабильным и находился в диапазоне 30 - 33,5.')


In [ ]:
find = re.compile("([^,=]+)")

for sent in sents(posts[i]):
    pprint(sent)
    pprint(" ".join(m.lemmatize(sent)))
    lemmatizing_words = []
#     pprint(json.dumps(m.analyze(sent), ensure_ascii=False))
    for token_analize in m.analyze(sent):
#         pprint(token_analize)
        if "analysis" in token_analize:
            lexem = token_analize["analysis"][0]["lex"]
            grammemes = token_analize["analysis"][0]["gr"]
            grammeme = re.search(find, grammemes).group(0)
            lemmatizing_words.append(f"{lexem}_{grammeme}")
    print(' '.join(lemmatizing_words), "\n\n")

In [129]:
text = "You should never modify something you are iterating over. This is not guaranteed to work in all cases. Depending on the data types, the iterator returns a copy and not a view, and writing to it will have no effect."


morphological_analyzer = Mystem()
stemmer = SnowballStemmer("english")

find = re.compile("([^,=]+)")

for sent in sents(text):
    pprint(sent)
    lemmatizing_words = []
    for token_analize in morphological_analyzer.analyze(sent):
#         pprint(token_analize)
        if "analysis" in token_analize:
            if token_analize["analysis"]:
                lexem = token_analize["analysis"][0]["lex"]
                grammemes = token_analize["analysis"][0]["gr"]
                grammeme = re.search(find, grammemes).group(0)
                lemmatizing_words.append(f"{lexem}_{grammeme}")
            else:
                for word in word_tokenize(token_analize["text"]):
                    lemmatizing_words.append(stemmer.stem(word))
    print(' '.join(lemmatizing_words), "\n\n")


'You should never modify something you are iterating over.'
you should never modifi someth you are iter over 


'This is not guaranteed to work in all cases.'
this is not guarante to work in all case 


('Depending on the data types, the iterator returns a copy and not a view, and '
 'writing to it will have no effect.')
depend on the data type the iter return a copi and not a view and write to it will have no effect 




In [ ]:
# Итоговый лемматизатор

In [175]:
morphological_analyzer = Mystem()
stemmer = SnowballStemmer("english")
stop_words = load_stop_words()
find = re.compile("([^,=]+)")

def lemmatize(text):
    lemmatizing_sents = []
    for para in paras(text):
        for sentence in sents(para):
            lemmatizing_words = []
            for token_analize in morphological_analyzer.analyze(sentence):
                if "analysis" in token_analize:
                    if token_analize["text"] in stop_words:
                        continue
                    if token_analize["analysis"]:
                        lexem = token_analize["analysis"][0]["lex"]
                        if lexem in stop_words:
                            continue
                        grammemes = token_analize["analysis"][0]["gr"]
                        grammeme = re.search(find, grammemes).group(0)
                        lemmatizing_words.append(f"{lexem}_{grammeme}")
                    else:
                        lexem = stemmer.stem(token_analize["text"])
                        if lexem in stop_words:
                            continue
                        lemmatizing_words.append(lexem)
            lemmatizing_sents.append(lemmatizing_words)
    return lemmatizing_sents

In [176]:
%%timeit

lemmatizing_sents = lemmatize(posts[i])

26.4 ms ± 852 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [178]:
lemmatizing_sents = lemmatize(posts[i])
lemmatizing_sents

[['здоровье_S'],
 ['система_S', 'емиас_S', 'зарегистрироваться_V', 'москвич_S'],
 ['единый_A',
  'медицинский_A',
  'информационный_A',
  'аналитический_A',
  'система_S',
  'емиас_S',
  'пользоваться_V',
  'москвич_S'],
 ['цифра_S',
  'называть_V',
  'мэр_S',
  'столица_S',
  'сергей_S',
  'собянин_S',
  'посещать_V',
  'ситуационный_A',
  'центр_S',
  'департамент_S',
  'здравоохранение_S'],
 ['помощь_S',
  'емиас_S',
  'легко_ADV',
  'находить_V',
  'близкий_A',
  'дом_S',
  'поликлиника_S',
  'получать_V',
  'справка_S',
  'льготный_A',
  'рецепт_S'],
 ['большинство_S',
  'горожанин_S',
  'слово_S',
  'градоначальник_S',
  'использовать_V',
  'система_S',
  'запись_S',
  'врач_S'],
 ['электронный_A',
  'запись_S',
  'большой_A',
  'подспорье_S',
  'врач_S',
  'возможность_S',
  'выписка_S',
  'электронный_A',
  'рецепт_S',
  'больничный_A',
  'лист_S',
  'отмечать_V',
  'мэр_S'],
 ['формироваться_V', 'сервис_S'],
 ['например_ADV',
  'электронный_A',
  'медицинский_A',
  'карта_S',


In [171]:
morphological_analyzer = Mystem()
stemmer = SnowballStemmer("english")
stop_words = load_stop_words()
find = re.compile("([^,=]+)")

def lemmatize(text):
    lemmatizing_sents = []
    for para in paras(text):
        for sentence in sents(para):
            pprint(sentence)
            lemmatizing_words = []
            for token_analize in morphological_analyzer.analyze(sentence):
                if "analysis" in token_analize:
                    if token_analize["text"] in stop_words:
                        continue
                    if token_analize["analysis"]:
                        lexem = token_analize["analysis"][0]["lex"]
                        if lexem in stop_words:
                            continue
                        grammemes = token_analize["analysis"][0]["gr"]
                        grammeme = re.search(find, grammemes).group(0)
                        lemmatizing_words.append(f"{lexem}_{grammeme}")
                    else:
                        lexem = stemmer.stem(token_analize["text"])
                        if lexem in stop_words:
                            continue
                        lemmatizing_words.append(lexem)
            lemmatizing_sents.append(lemmatizing_words)
    return lemmatizing_sents

In [172]:
pprint(lemmatize(posts[i]))

'ЗДОРОВЬЕ'
'В системе ЕМИАС зарегистрировались 75% москвичей'
('ЕДИНОЙ медицинской информационно-аналитической системой (ЕМИАС) пользуются '
 'около 9 миллионов, или 75%, москвичей.')
('Эту цифру назвал мэр столицы Сергей Собянин, посетив ситуационный центр '
 'департамента здравоохранения.')
('С помощью ЕМИАС сейчас можно легко найти ближайшую к дому поликлинику, '
 'получить справки и льготные рецепты.')
('При этом большинство горожан, по словам градоначальника, используют систему '
 'для записи к врачу.')
('«Электронная запись - большое подспорье для людей, а для самих врачей - '
 'возможность выписки электронных рецептов и больничных листов, - отметил мэр.')
'- Сегодня формируются и другие сервисы.'
('Например, электронная медицинская карта, которая помогает руководителям '
 'поликлиник управлять потоком пациентов, решать проблемы очередей и '
 'неравномерного распределения врачей».')
('Устранить недочеты призван и ситуационный центр департамента '
 'здравоохранения, куда в режиме 

In [133]:
morphological_analyzer = Mystem()
stemmer = SnowballStemmer("english")
find = re.compile("([^,=]+)")

def lemmatize(text):
    lemmatizing_sents = []
    for para in paras(text):
        for sentence in sents(para):
            pprint(sentence)
            lemmatizing_words = []
            for token_analize in morphological_analyzer.analyze(sentence):
                if "analysis" in token_analize:
                    if token_analize["analysis"]:
                        lexem = token_analize["analysis"][0]["lex"]
                        grammemes = token_analize["analysis"][0]["gr"]
                        grammeme = re.search(find, grammemes).group(0)
                        lemmatizing_words.append(f"{lexem}_{grammeme}")
                    else:
                        lemmatizing_words.append(stemmer.stem(token_analize["text"]))
            lemmatizing_sents.append(lemmatizing_words)
    return lemmatizing_sents

In [134]:
pprint(lemmatize(posts[i]))

'ЗДОРОВЬЕ'
'В системе ЕМИАС зарегистрировались 75% москвичей'
('ЕДИНОЙ медицинской информационно-аналитической системой (ЕМИАС) пользуются '
 'около 9 миллионов, или 75%, москвичей.')
('Эту цифру назвал мэр столицы Сергей Собянин, посетив ситуационный центр '
 'департамента здравоохранения.')
('С помощью ЕМИАС сейчас можно легко найти ближайшую к дому поликлинику, '
 'получить справки и льготные рецепты.')
('При этом большинство горожан, по словам градоначальника, используют систему '
 'для записи к врачу.')
('«Электронная запись - большое подспорье для людей, а для самих врачей - '
 'возможность выписки электронных рецептов и больничных листов, - отметил мэр.')
'- Сегодня формируются и другие сервисы.'
('Например, электронная медицинская карта, которая помогает руководителям '
 'поликлиник управлять потоком пациентов, решать проблемы очередей и '
 'неравномерного распределения врачей».')
('Устранить недочеты призван и ситуационный центр департамента '
 'здравоохранения, куда в режиме 

In [73]:
find = re.compile("([^,=]+)")

In [74]:
st = "S,сред,неод=(пр,ед|вин,ед|им,ед)"
re.search(find, st).group(0)

'S'

In [76]:
st = "V,сов,нп=прош,мн,изъяв"
re.search(find, st).group(0)

'V'

In [79]:
st = "PR="
re.search(find, st).group(0)

'PR'

In [78]:
st = "A=(пр,ед,полн,жен|дат,ед,полн,жен|род,ед,полн,жен|твор,ед,полн,жен)"
re.search(find, st).group(0)

'A'

In [80]:
st = "CONJ="
re.search(find, st).group(0)

'CONJ'

In [49]:
for sent in sents(posts[i]):
    pprint(" ".join(m.lemmatize(sent)))
    pprint(m.analyze(sent))
    print("\n\n")

'здоровье \n'
[{'analysis': [{'gr': 'S,сред,неод=(пр,ед|вин,ед|им,ед)',
                'lex': 'здоровье',
                'wt': 1}],
  'text': 'ЗДОРОВЬЕ'},
 {'text': '\n'}]



'в   система   емиас   зарегистрироваться   75 %  москвич \n'
[{'analysis': [{'gr': 'PR=', 'lex': 'в', 'wt': 0.9999917878}], 'text': 'В'},
 {'text': ' '},
 {'analysis': [{'gr': 'S,жен,неод=(пр,ед|дат,ед)', 'lex': 'система', 'wt': 1}],
  'text': 'системе'},
 {'text': ' '},
 {'analysis': [{'gr': 'S,муж,неод=(вин,ед|им,ед)',
                'lex': 'емиас',
                'qual': 'bastard',
                'wt': 0.4180647166}],
  'text': 'ЕМИАС'},
 {'text': ' '},
 {'analysis': [{'gr': 'V,сов,нп=прош,мн,изъяв',
                'lex': 'зарегистрироваться',
                'wt': 1}],
  'text': 'зарегистрировались'},
 {'text': ' '},
 {'text': '75'},
 {'text': '% '},
 {'analysis': [{'gr': 'S,муж,од=(вин,мн|род,мн)',
                'lex': 'москвич',
                'wt': 0.950734991}],
  'text': 'москвичей'},
 {'text': 

In [17]:
%%timeit

doc = []
for sent in sents(posts[i]):
    lemmas = m.lemmatize(sent)
    doc.append(lemmas)

26.5 ms ± 154 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
doc = []
for sent in sents(posts[i]):
    lemmas = m.lemmatize(sent)
    doc.append(lemmas)
pprint(doc)

[['здоровье', '\n'],
 ['в',
  ' ',
  'система',
  ' ',
  'емиас',
  ' ',
  'зарегистрироваться',
  ' ',
  '75',
  '% ',
  'москвич',
  '\n'],
 ['единый',
  ' ',
  'медицинский',
  ' ',
  'информационный',
  '-',
  'аналитический',
  ' ',
  'система',
  ' (',
  'емиас',
  ') ',
  'пользоваться',
  ' ',
  'около',
  ' ',
  '9',
  ' ',
  'миллион',
  ', ',
  'или',
  ' ',
  '75',
  '%, ',
  'москвич',
  '.',
  '\n'],
 ['этот',
  ' ',
  'цифра',
  ' ',
  'называть',
  ' ',
  'мэр',
  ' ',
  'столица',
  ' ',
  'сергей',
  ' ',
  'собянин',
  ', ',
  'посещать',
  ' ',
  'ситуационный',
  ' ',
  'центр',
  ' ',
  'департамент',
  ' ',
  'здравоохранение',
  '.',
  '\n'],
 ['с',
  ' ',
  'помощь',
  ' ',
  'емиас',
  ' ',
  'сейчас',
  ' ',
  'можно',
  ' ',
  'легко',
  ' ',
  'находить',
  ' ',
  'близкий',
  ' ',
  'к',
  ' ',
  'дом',
  ' ',
  'поликлиника',
  ', ',
  'получать',
  ' ',
  'справка',
  ' ',
  'и',
  ' ',
  'льготный',
  ' ',
  'рецепт',
  '.',
  '\n'],
 ['при',
  ' ',
  '